In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

In [3]:
num = 1000
event_time = int(num/2)
R_market = np.random.normal(0, 1, num) + np.arange(num) / num
R_target = 2 + R_market + np.random.normal(0, 1, num) + (np.arange(num) == int(num/2) + 1) * 2

results = sm.OLS(R_target[:event_time], sm.add_constant(R_market[:event_time])).fit()

alpha, beta = results.params

resid = R_target - results.predict(sm.add_constant(R_market))
print(resid[event_time + 1] / resid[:event_time].std(ddof=2))

1.1260965414520492


In [4]:
num = 1000
event_time = int(num/2)

np.random.seed(0)
R_market = np.random.normal(0, 1, num) + np.arange(num) / num
R_target = 2 + R_market + np.random.normal(0, 1, num) + (np.arange(num) == int(num/2) + 1) * 2

detected = []
for t in range(100, num - 2):
    np.random.seed(0)
    results = sm.OLS(R_target[:t], sm.add_constant(R_market[:t])).fit()
    resid = R_target - results.predict(sm.add_constant(R_market))
    std_resid = resid[:t].std(ddof=2)
    test_stat = resid[t+1] / std_resid
    
    if abs(test_stat) > 1.96:
        detected.append(1)
    else:
        detected.append(0)

np.mean(detected)

np.float64(0.045657015590200446)

In [5]:
def make_error(corr_const, num):
    sigma = 5 * 1 / np.sqrt((1 - corr_const)**2 / (1 - corr_const**2))
    err = list()
    prev = np.random.normal(0, sigma)
    
    for n in range(num):
        prev = corr_const * prev + (1 - corr_const) * np.random.normal(0, sigma)
        err.append(prev)
    
    return np.array(err)

In [6]:
num = 1000
event_time = int(num/2)
detected = []

np.random.seed(0)
R_market = np.random.normal(0, 1, num) + np.arange(num) / num

error = make_error(corr_const=0.9, num=num)
R_target = 2 + R_market + error + (np.arange(num) == int(num/2) + 1) * 2

for t in range(100, num - 2):
    np.random.seed(0)
    results = sm.OLS(R_target[:t], sm.add_constant(R_market[:t])).fit()
    resid = R_target - results.predict(sm.add_constant(R_market))
    std_resid = resid[:t].std(ddof=2)
    test_stat = resid[t+1] / std_resid
    detected.append(abs(test_stat) > 1.96)
    
np.mean(detected)

np.float64(0.0378619153674833)